In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

In [3]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [4]:
nlp = spacy.load('en')

# Load dataset

In [5]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [6]:
# number of documents
len(docs)

18846

In [7]:
# store an index with a doc
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [8]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

100%|██████████| 18846/18846 [00:44<00:00, 419.37it/s]


number of removed short documents: 3985
total number of tokens: 1439861
number of tokens to be removed: 393091
number of additionally removed short documents: 2032
total number of tokens: 1023189

minimum word count number: 14
this number can be less than MIN_COUNTS because of document removal


In [9]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [10]:
data = []
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

12829it [00:02, 4599.89it/s]


In [11]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [12]:
# number of windows (equals to the total number of tokens)
data.shape[0]

1023189

# Get unigram distribution

In [13]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [14]:
%%time
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=64, window=5, workers=4, sg=1, negative=15, iter=40)

vocab_size = len(decoder)
embedding_dim = 64
word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')

for i in decoder:
    word_vectors[i] = model.wv[str(i)]

CPU times: user 17min 14s, sys: 168 ms, total: 17min 14s
Wall time: 4min 19s


In [ ]:
# number of unique words
vocab_size

# Prepare initialization for document weights

In [15]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [16]:
%%time
n_topics = 20
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

CPU times: user 1min 7s, sys: 856 ms, total: 1min 7s
Wall time: 37.2 s


In [17]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : jesus bible lord sin christ word christian father psalms prayer
topic 1 : team play win season hockey league fan score goal player
topic 2 : health rate increase medical study tax food age gun report
topic 3 : cost job scsi ide money bob billion team station space
topic 4 : christ church jesus child mary sin greek john book aids
topic 5 : chip ripem encryption public security des message bit clipper pgp
topic 6 : card disk board driver hard windows memory machine software mac
topic 7 : church belief evidence claim religion exist word christian reason argument
topic 8 : version color software user ftp format text tool widget display
topic 9 : satellite spacecraft launch bos mission space det tor period earth
topic 10 : window application code display server software user unix object version
topic 11 : monitor card offer sell san sale video condition diamond old
topic 12 : price sell service technology buy computer market sale development pay
topic 13 : israel israeli arab grou

In [18]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 12829/12829 [00:23<00:00, 535.42it/s]


# Save data

In [19]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)